# Handwriting Recognition on MNIST Dataset

# Imports

In [393]:
from numpy.random import seed
seed(888)
from tensorflow import set_random_seed
set_random_seed(404)

In [481]:
import numpy as np
import tensorflow as tf
import os

from PIL import Image
from time import strftime

# Constants

In [395]:
x_train_path = 'C:/Users/akars/Desktop/Studies/EXTRA/ML/Handwriting Classification/digit_xtrain.csv'
x_test_path = 'C:/Users/akars/Desktop/Studies/EXTRA/ML/Handwriting Classification/digit_xtest.csv'
y_train_path = 'C:/Users/akars/Desktop/Studies/EXTRA/ML/Handwriting Classification/digit_ytrain.csv'
y_test_path = 'C:/Users/akars/Desktop/Studies/EXTRA/ML/Handwriting Classification/digit_ytest.csv'

In [396]:
num_classes = 10
validation_size = 10000
image_width = 28
image_height = 28
channels = 1
total_inputs = image_width*image_height*channels

logging_path = 'tensorboard_mnist_digit_logs/'

# Get the Data

In [397]:
%%time

y_train = np.loadtxt(y_train_path, delimiter=',', dtype=int)

Wall time: 341 ms


In [398]:
y_test = np.loadtxt(y_test_path, delimiter=',', dtype=int)

In [399]:
%%time
x_train = np.loadtxt(x_train_path, delimiter=',', dtype=int)

Wall time: 53.5 s


In [535]:
%%time
x_test = np.loadtxt(x_test_path, delimiter=',', dtype=int)

Wall time: 9.06 s


# Explore

In [401]:
x_train.shape

(60000, 784)

In [402]:
x_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [403]:
y_train.shape

(60000,)

In [536]:
x_test.shape

(10000, 784)

In [405]:
y_train[:5]

array([5, 0, 4, 1, 9])

# Data Preprocessing

In [406]:
# Re-scale Features
x_train, x_test = x_train/255.0, x_test/255.0

#### Convert target values to one-hot encoding

In [407]:
y_train = np.eye(num_classes)[y_train]

In [408]:
y_train.shape

(60000, 10)

In [409]:
y_test = np.eye(num_classes)[y_test]

In [410]:
y_test.shape

(10000, 10)

#### Create validation dataset from training data

In [411]:
x_val = x_train[:validation_size]
y_val = y_train[:validation_size]

In [412]:
x_train = x_train[validation_size:]
y_train = y_train[validation_size:]

 # Setup Tensorflow Graph

In [492]:
X = tf.placeholder(tf.float32, shape=(None,total_inputs), name='X')
Y = tf.placeholder(tf.float32, shape=(None, num_classes), name='labels')

### Neural Network Architecture

#### Hyperparameters

In [493]:
num_epochs = 50
learning_rate = 1e-3

n_hidden1 = 512
n_hidden2 = 64

In [494]:
def setup_layer(input, weight_dim, bias_dim, name):
    
    with tf.name_scope(name):
        initial_w = tf.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        w = tf.Variable(initial_value=initial_w, name='W')

        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=initial_b, name='B')

        layer_in = tf.matmul(input, w) + b
        
        if name == 'out':
            layer_out = tf.nn.softmax(layer_in)
        else:
            layer_out = tf.nn.relu(layer_in)
        
        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)
        
        return layer_out
            

In [495]:
# Model without dropout
# layer_1 = setup_layer(X, [total_inputs, n_hidden1], 
#                       [n_hidden1], 'layer_1')

# layer_2 = setup_layer(layer_1, [n_hidden1, n_hidden2], 
#                       [n_hidden2], 'layer_2')

# output = setup_layer(layer_2, [n_hidden2, num_classes], 
#                       [num_classes], 'out')

# model_name = f'{n_hidden1}-{n_hidden2} LR{learning_rate} E{num_epochs}'

In [496]:
layer_1 = setup_layer(X, [total_inputs, n_hidden1], 
                      [n_hidden1], 'layer_1')

layer_drop = tf.nn.dropout(layer_1, keep_prob=0.8, name='dropout_layer')

layer_2 = setup_layer(layer_drop, [n_hidden1, n_hidden2], 
                      [n_hidden2], 'layer_2')

output = setup_layer(layer_2, [n_hidden2, num_classes], 
                      [num_classes], 'out')

model_name = f'{n_hidden1}-DO-{n_hidden2} LR{learning_rate} E{num_epochs}'

# Tensorboard Setup

In [497]:
# Folder for Tensorboard

folder_name = f'{model_name} at {strftime("%H %M")}'
directory = os.path.join(logging_path, folder_name)

try:
    os.makedirs(directory)
except OSError as err:
    print(err.strerror)
else:
    print("Successfully Created Directory")

Successfully Created Directory


# Loss, Optimization & Metrics

#### Defining loss function

In [498]:
with tf.name_scope('loss_calc'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output))

#### Defining optimizer

In [499]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_step = optimizer.minimize(loss)

#### Accuracy Metric

In [500]:
with tf.name_scope('accuracy_calc'):
    correct_pred = tf.equal(tf.argmax(output, axis=1), tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [501]:
with tf.name_scope('performance'):
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('cost', loss)

#### Check Input Images in Tensorboard

In [502]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X, [-1, 28, 28, 1])
    tf.summary.image('image_input', x_image, max_outputs=4)

# Run Session

In [503]:
sess = tf.Session()

#### Setup Filewriter and Merge Summaries

In [504]:
merged_summary = tf.summary.merge_all()

train_writer = tf.summary.FileWriter(directory + '/train')
train_writer.add_graph(sess.graph)

validation_writer = tf.summary.FileWriter(directory + '/validation')

#### Initialize all the variables

In [505]:
init = tf.global_variables_initializer()
sess.run(init)

### Batching the Data

In [506]:
size_of_batch = 1000

num_examples = y_train.shape[0]
num_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0

In [507]:
def next_batch(batch_size, data, labels):
    
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
        
    end = index_in_epoch
    
    return data[start:end], labels[start:end]

#### Training Loop

In [508]:
for epoch in range(num_epochs):    
    for i in range(num_iterations):
     # Training   
        batch_x, batch_y = next_batch(size_of_batch, x_train, y_train)
        
        feed_dictionary = {X:batch_x, Y:batch_y}
        
        sess.run(train_step, feed_dictionary)
        
    s, batch_acc = sess.run(fetches=[merged_summary, accuracy], feed_dict = feed_dictionary)
        
    train_writer.add_summary(s, epoch)
    
    print(f'Epoch {epoch} \t| Training Accuracy = {batch_acc}')
    # Validation
    summary = sess.run(fetches=merged_summary, feed_dict ={X:x_val, Y:y_val})
    validation_writer.add_summary(summary, epoch)

    
print('Done training')

Epoch 0 	| Training Accuracy = 0.8539999723434448
Epoch 1 	| Training Accuracy = 0.8610000014305115
Epoch 2 	| Training Accuracy = 0.8650000095367432
Epoch 3 	| Training Accuracy = 0.8709999918937683
Epoch 4 	| Training Accuracy = 0.871999979019165
Epoch 5 	| Training Accuracy = 0.9670000076293945
Epoch 6 	| Training Accuracy = 0.9729999899864197
Epoch 7 	| Training Accuracy = 0.9739999771118164
Epoch 8 	| Training Accuracy = 0.9819999933242798
Epoch 9 	| Training Accuracy = 0.984000027179718
Epoch 10 	| Training Accuracy = 0.9850000143051147
Epoch 11 	| Training Accuracy = 0.9860000014305115
Epoch 12 	| Training Accuracy = 0.9869999885559082
Epoch 13 	| Training Accuracy = 0.9879999756813049
Epoch 14 	| Training Accuracy = 0.9869999885559082
Epoch 15 	| Training Accuracy = 0.9860000014305115
Epoch 16 	| Training Accuracy = 0.9890000224113464
Epoch 17 	| Training Accuracy = 0.9879999756813049
Epoch 18 	| Training Accuracy = 0.9900000095367432
Epoch 19 	| Training Accuracy = 0.990000009

# Make a Prediction

In [509]:
img = Image.open('C:/Users/akars/Desktop/Studies/EXTRA/ML/Handwriting Classification/test_img.png')
img

In [510]:
bw = img.convert('L')

In [511]:
img_arr = np.invert(bw)

In [512]:
img_arr.shape

(28, 28)

In [513]:
test_img = img_arr.ravel()

In [514]:
test_img.shape

(784,)

In [518]:
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X:[test_img]})

In [519]:
print(f'Prediction for test image is {prediction}')

Prediction for test image is [2]


# Testing and Evaluation

In [537]:
test_acc = sess.run(fetches=accuracy, feed_dict={X:x_test, Y:y_test})
print(f'Accuracy on test set is {test_acc:0.2%}')

Accuracy on test set is 97.84%


# Reset for the Next Run

In [ ]:
train_writer.close()
validation_writer.close()
sess.close()
tf.reset_default_graph()

# Code initally before refractoring

#### First hidden layer

In [ ]:
# with tf.name_scope('hidden_1'):    
#     initial_w1 = tf.truncated_normal(shape=[total_inputs, n_hidden1], stddev=0.1, seed=42)
#     w1 = tf.Variable(initial_value=initial_w1, name = 'w1')

#     initial_b1 = tf.constant(value=0.0, shape=[n_hidden1])
#     b1 = tf.Variable(initial_value=initial_b1, name='b1')

#     layer1_in = tf.matmul(X, w1) + b1
#     layer1_out = tf.nn.relu(layer1_in)

In [ ]:
# initial_w1

#### Second hidden layer

In [ ]:
# with tf.name_scope('hidden_2'):
#     initial_w2 = tf.truncated_normal(shape=[n_hidden1, n_hidden2], stddev=0.1, seed=42)
#     w2 = tf.Variable(initial_value=initial_w2, name='w2')

#     initial_b2 = tf.constant(value=0.0, shape=[n_hidden2])
#     b2 = tf.Variable(initial_value=initial_b2, name='wb')

#     layer2_in = tf.matmul(layer1_out, w2) + b2
#     layer2_out = tf.nn.relu(layer2_in)

#### Output layer

In [ ]:
# with tf.name_scope('output_layer'):
#     initial_w3 = tf.truncated_normal(shape=[n_hidden2, num_classes], stddev=0.1, seed=42)
#     w3 = tf.Variable(initial_value=initial_w3, name='w3')

#     initial_b3 = tf.constant(value=0.0, shape=[num_classes])
#     b3 = tf.Variable(initial_value=initial_b3, name='b3')

#     layer3_in = tf.matmul(layer2_out, w3) + b3
#     output = tf.nn.softmax(layer3_in)

In [ ]:
# w1.eval(sess)

In [ ]:
# b3.eval(sess)